<a href="https://colab.research.google.com/github/hfnvbh/colab-trt-pose/blob/master/trt-pose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install torch torchvision

In [3]:
!wget http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!apt-key add /var/cuda-repo-10-2-local-10.2.89-440.33.01/7fa2af80.pub
!apt-get update
!apt-get -y install cuda-10-2

--2020-06-09 07:36:13--  http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1896270068 (1.8G) [application/x-deb]
Saving to: ‘cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb’

cuda-repo-ubuntu180 100%[===================>]   1.77G  38.5MB/s    in 14s     

2020-06-09 07:36:26 (132 MB/s) - ‘cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb’ saved [1896270068/1896270068]

Selecting previously unselected package cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01.
(Reading database ... 144467 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.d

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


In [6]:
!wget -q https://www.dropbox.com/s/sprhhdox8w84ph4/nv-tensorrt-repo-ubuntu1804-cuda10.2-trt7.0.0.11-ga-20191216_1-1_amd64.deb
!dpkg -i nv-tensorrt-repo-ubuntu1804-cuda10.2-trt7.0.0.11-ga-20191216_1-1_amd64.deb
!apt-key add /var/nv-tensorrt-repo-cuda10.2-trt7.0.0.11-ga-20191216/7fa2af80.pub
!apt-get update
!apt-get install tensorrt

Selecting previously unselected package nv-tensorrt-repo-ubuntu1804-cuda10.2-trt7.0.0.11-ga-20191216.
(Reading database ... 156968 files and directories currently installed.)
Preparing to unpack nv-tensorrt-repo-ubuntu1804-cuda10.2-trt7.0.0.11-ga-20191216_1-1_amd64.deb ...
Unpacking nv-tensorrt-repo-ubuntu1804-cuda10.2-trt7.0.0.11-ga-20191216 (1-1) ...
Setting up nv-tensorrt-repo-ubuntu1804-cuda10.2-trt7.0.0.11-ga-20191216 (1-1) ...
OK


In [7]:

!a#pt-get install python3-libnvinfer-dev
!ap#t-get install uff-converter-tf

Get:1 file:/var/cuda-repo-10-2-local-10.2.89-440.33.01  InRelease
Ign:1 file:/var/cuda-repo-10-2-local-10.2.89-440.33.01  InRelease
Get:2 file:/var/nv-tensorrt-repo-cuda10.2-trt7.0.0.11-ga-20191216  InRelease
Ign:2 file:/var/nv-tensorrt-repo-cuda10.2-trt7.0.0.11-ga-20191216  InRelease
Get:3 file:/var/cuda-repo-10-2-local-10.2.89-440.33.01  Release [574 B]
Get:4 file:/var/nv-tensorrt-repo-cuda10.2-trt7.0.0.11-ga-20191216  Release [574 B]
Get:3 file:/var/cuda-repo-10-2-local-10.2.89-440.33.01  Release [574 B]
Get:4 file:/var/nv-tensorrt-repo-cuda10.2-trt7.0.0.11-ga-20191216  Release [574 B]
Get:5 file:/var/nv-tensorrt-repo-cuda10.2-trt7.0.0.11-ga-20191216  Release.gpg [836 B]
Get:5 file:/var/nv-tensorrt-repo-cuda10.2-trt7.0.0.11-ga-20191216  Release.gpg [836 B]
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-le

In [8]:
!dpkg -l | grep TensorRT

ii  libnvinfer-bin                                               7.0.0-1+cuda10.2                                  amd64        TensorRT binaries
ii  libnvinfer-dev                                               7.0.0-1+cuda10.2                                  amd64        TensorRT development libraries and headers
ii  libnvinfer-doc                                               7.0.0-1+cuda10.2                                  all          TensorRT documentation
ii  libnvinfer-plugin-dev                                        7.0.0-1+cuda10.2                                  amd64        TensorRT plugin libraries
ii  libnvinfer-plugin7                                           7.0.0-1+cuda10.2                                  amd64        TensorRT plugin libraries
ii  libnvinfer-samples                                           7.0.0-1+cuda10.2                                  all          TensorRT samples
ii  libnvinfer7                                                  7.0.0-1+cuda10

In [9]:
!pip3 install tqdm cython pycocotools
!apt-get install python3-matplotlib
!git clone https://github.com/NVIDIA-AI-IOT/trt_pose
!cd trt_pose && python3 setup.py install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-lyx javascript-common libjs-jquery libjs-jquery-ui
  python-matplotlib-data python3-cycler python3-dateutil python3-olefile
  python3-pil python3-pyparsing python3-six python3-tz ttf-bitstream-vera
Suggested packages:
  apache2 | lighttpd | httpd libjs-jquery-ui-docs python-cycler-doc dvipng
  gir1.2-gtk-3.0 ghostscript inkscape ipython3 python-matplotlib-doc
  python3-cairocffi python3-gi-cairo python3-gobject python3-nose
  python3-pyqt4 python3-scipy python3-sip python3-tornado texlive-extra-utils
  texlive-latex-extra ttf-staypuft python-pil-doc python3-pil-dbg
  python-pyparsing-doc
The following NEW packages will be installed:
  fonts-lyx javascript-common libjs-jquery libjs-jquery-ui
  python-matplotlib-data python3-cycler python3-dateutil python3-matplotlib
  python3-olefile python3-pil python3-pyparsing python3-six python3

In [0]:
!wget -q https://www.dropbox.com/s/r7ex76ofqsnjkjy/resnet18_baseline_att_224x224_A_epoch_249.pth

In [0]:
import sys
sys.path.append('trt_pose')

In [13]:

import json
import trt_pose.coco

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

ModuleNotFoundError: ignored

In [0]:

import trt_pose.models